# Steady State Diffusion in a Composite Bar using PINNs

In this notebook, we will solve the steady state 1-dimensional heat transfer in a composite bar and also use a Parameterized approach for the same.

#### Contents of the Notebook
- [Steady State 1D Diffusion in a Composite Bar using PINNs](#Steady-State-1D-Diffusion-in-a-Composite-Bar-using-PINNs)
    - [Problem Description](#Problem-Description)
    - [Case Setup](#Case-Setup)
    - [Step 1: Creating the geometry](#Step-1:-Creating-the-geometry)
    - [Step 2: Defining the PDEs and creating the nodes](#Step-2:-Defining-the-PDEs-and-creating-the-nodes)
    - [Step 3: Setting up the domain and assigning the boundary and PDE constraints](#Step-3:-Setting-up-the-domain-and-assigning-the-boundary-and-PDE-constraints)
    - [Step 4: Adding Validators, Inferencers and Monitors](#Step-4:-Adding-Validators,-Inferencers-and-Monitors)
    - [Step 5: Hydra configuration](#Step-5:-Hydra-configuration)
    - [Step 6: Putting everything together: Solver and training](#Step-6:-Putting-everything-together:-Solver-and-training)
    - [Visualizing the solution](#Visualizing-the-solution)
- [Parameterized 1D Diffusion of Composite Bar](#Parameterized-1D-Diffusion-of-Composite-Bar)
    - [Case Setup - Parameterized](#Case-Setup---Parameterized)
    - [Step 1: Creating the geometry - Parameterized](#Step-1:-Creating-the-geometry---Parameterized)
    - [Step 2: Adding Parameterized PDE and Neural Network nodes](#Step-2:-Adding-Parameterized-PDE-and-Neural-Network-nodes)
    - [Step 3: Adding Parameterized boundary and PDE Constraints](#Step-3:-Adding-Parameterized-boundary-and-PDE-Constraints)
    - [Step 4: Adding Validators and Monitors](#Step-4:-Adding-Validators-and-Monitors)
    - [Step 5: Hydra configuration - Parameterized](#Step-5:-Hydra-configuration---Parameterized)
    - [Step 6: Solver and Training](#Step-6:-Solver-and-Training)
    - [Visualizing the solution - Parameterized](#Visualizing-the-solution---Parameterized)
    

#### Learning Outcomes
- How to use the Constructive Solid Geometry (CSG) module
- How to use Modulus to solve a parameterized PDEs

## Steady State 1D Diffusion in a Composite Bar using PINNs

### Problem Description

Our aim is to obtain the temperature distribution inside the bar that is made up of two materials with different thermal conductivity. The geometry and the problem specification of the problem can be seen below

<center><img src="images/diffusion_bar_geometry.png" alt="Drawing" style="width: 600px;"/></center>

The composite bar extends from $x=0$ to $x=2$. The bar has material of conductivity $D_1=10$ from $x=0$ to $x=1$ and $D_2=0.1$ from $x=1$ to $x=2$. Both the ends of the bar, $x=0$ and $x=2$ are maintained at a constant temperatures of $0$ and $100$ respectively. For simplicity of modelling, we will treat the composite bar as two separate bars, bar 1 and bar 2, whose ends are joined together. We will treat the temperatures in the bar 1 as $U_1$ and the temperature in bar 2 as $U_2$. 

The equations and boundary conditions governing the problem can be mathematically expressed as

One dimensional diffusion of temperature in bar 1 and 2:

$$
\begin{align}
\frac{d}{dx}\left( D_1\frac{dU_1}{dx} \right) = 0, && \text{when } 0<x<1 \\
\frac{d}{dx}\left( D_2\frac{dU_2}{dx} \right) = 0, && \text{when } 1<x<2 \\
\end{align}
$$

Flux and temperature continuity at interface $(x=1)$
$$
\begin{align}
D_1\frac{dU_1}{dx} = D_2\frac{dU_2}{dx}, && \text{when } x=1 \\
U_1 = U_2, && \text{when } x=1 \\
\end{align}
$$


### Case Setup

Now let's start the problem by importing the required libraries and packages

#### Note : In this notebook we will describe the contents of the [`diffusion_bar.py`](../../source_code/diffusion_1d/diffusion_bar.py) script

```python
import torch
import numpy as np
from sympy import Symbol, Eq, Function, Number

import modulus
from modulus.hydra import ModulusConfig, instantiate_arch
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Line1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.domain.monitor import PointwiseMonitor
from modulus.key import Key
from modulus.node import Node
from modulus.eq.pde import PDE
```

### Step 1: Creating the geometry

In this problem, we will create the 1-dimensional geometry using `Line1D` class from the geometry module. The module also contains several 2d and 3d shapes like rectangle, circle, triangle, cuboids, sphere, torus, cones, tetrahedrons, etc. We will define the one dimensional line object using the two end-points. For the composite bar, we will create two separate bars as defined in the problem statement

```python
# params for domain
L1 = Line1D(0,1)
L2 = Line1D(1,2)
```

Next, we will define the properties for the problem statement, which will later be used while making the equations, boundary conditions, etc. Also, for this problem, we can find the temperature at the interface analytically, and we will use that to form validation data to compare our neural network results 

```python
D1 = 1e1
D2 = 1e-1

Tc = 100
Ta = 0
Tb = (Tc + (D1 / D2) * Ta) / (1 + (D1 / D2))

print(Ta)
print(Tb)
print(Tc)
```

### Step 2: Defining the PDEs and creating the nodes

Let's create the PDE to define the diffusion equation. We will define the equation in its most generic, transient 3-dimensional, form and then have an argument `dim` that can reduce it to lower dimensional forms. 
$$\frac{\partial T}{\partial t}= \nabla\cdot \left( D \nabla T \right) + Q$$

Let's start defining the equation by inhereting from the `PDE` class. We will create the initialization method for this class that defines the equation(s) of interest. We will be defining the diffusion equation using the source(`Q`), diffusivity(`D`), symbol for diffusion(`T`). If `D` or `Q` is given as a string, we will convert it to functional form. This will allow us to solve problems with spatially/temporally varying properties. 

```python
class Diffusion(PDE):
    name = "Diffusion"

    def __init__(self, T="T", D="D", Q=0, dim=3, time=True):
        # set params
        self.T = T
        self.dim = dim
        self.time = time

        # coordinates
        x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

        # time
        t = Symbol("t")

        # make input variables
        input_variables = {"x": x, "y": y, "z": z, "t": t}
        if self.dim == 1:
            input_variables.pop("y")
            input_variables.pop("z")
        elif self.dim == 2:
            input_variables.pop("z")
        if not self.time:
            input_variables.pop("t")

        # Temperature
        assert type(T) == str, "T needs to be string"
        T = Function(T)(*input_variables)

        # Diffusivity
        if type(D) is str:
            D = Function(D)(*input_variables)
        elif type(D) in [float, int]:
            D = Number(D)

        # Source
        if type(Q) is str:
            Q = Function(Q)(*input_variables)
        elif type(Q) in [float, int]:
            Q = Number(Q)

        # set equations
        self.equations = {}
        self.equations["diffusion_" + self.T] = (
            T.diff(t)
            - (D * T.diff(x)).diff(x)
            - (D * T.diff(y)).diff(y)
            - (D * T.diff(z)).diff(z)
            - Q
        )
```

First, we defined the input variables $x, y, z$ and $t$ with Sympy symbols. Then we defined the functions for $T$, $D$ and $Q$ that are dependent on the input variables $(x, y, z, t)$. Using these, we can write out our simple equation $T_t = \nabla \cdot (D \nabla T) + Q$. We store this equation in the class by adding it to the dictionary of `equations`.

Note that we moved all the terms of the PDE either to LHS or RHS. This way, while using the equations in the constraints, we
can assign a custom source function to the <code>’diffusion_T’</code> key instead of 0 to add more source terms to our PDE.
Now that we have defined our PDE, you can also bundle multiple PDEs together in the same file by adding new keys to the equations dictionary. Below we show the code for the interface boundary condition where we need to maintain the field (dirichlet) and flux (neumann) continuity.
<strong>Note:</strong> The field continuity condition is needed because we are solving for two different temperatures in the two bars. 


```python
class DiffusionInterface(PDE):
    name = "DiffusionInterface"

    def __init__(self, T_1, T_2, D_1, D_2, dim=3, time=True):
        # set params
        self.T_1 = T_1
        self.T_2 = T_2
        self.dim = dim
        self.time = time

        # coordinates
        x, y, z = Symbol("x"), Symbol("y"), Symbol("z")
        normal_x, normal_y, normal_z = (
            Symbol("normal_x"),
            Symbol("normal_y"),
            Symbol("normal_z"),
        )

        # time
        t = Symbol("t")

        # make input variables
        input_variables = {"x": x, "y": y, "z": z, "t": t}
        if self.dim == 1:
            input_variables.pop("y")
            input_variables.pop("z")
        elif self.dim == 2:
            input_variables.pop("z")
        if not self.time:
            input_variables.pop("t")

        # Diffusivity
        if type(D_1) is str:
            D_1 = Function(D_1)(*input_variables)
        elif type(D_1) in [float, int]:
            D_1 = Number(D_1)
        if type(D_2) is str:
            D_2 = Function(D_2)(*input_variables)
        elif type(D_2) in [float, int]:
            D_2 = Number(D_2)

        # variables to match the boundary conditions (example Temperature)
        T_1 = Function(T_1)(*input_variables)
        T_2 = Function(T_2)(*input_variables)

        # set equations
        self.equations = {}
        self.equations["diffusion_interface_dirichlet_" + self.T_1 + "_" + self.T_2] = (
            T_1 - T_2
        )
        flux_1 = D_1 * (
            normal_x * T_1.diff(x) + normal_y * T_1.diff(y) + normal_z * T_1.diff(z)
        )
        flux_2 = D_2 * (
            normal_x * T_2.diff(x) + normal_y * T_2.diff(y) + normal_z * T_2.diff(z)
        )
        self.equations["diffusion_interface_neumann_" + self.T_1 + "_" + self.T_2] = (
            flux_1 - flux_2
        )
```

#### Creating Neural Network nodes

The default `FullyConnectedArch` represents a 6 layer MLP (multi-layer perceptron) architecture with each layer containing 512 nodes and uses swish as the activation function. We will be using a layer size of 256 for our case.

We will define all the code for the problem in the `run` function as shown below. 

```python
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:

    # make list of nodes to unroll graph on
    diff_u1 = Diffusion(T="u_1", D=D1, dim=1, time=False)
    diff_u2 = Diffusion(T="u_2", D=D2, dim=1, time=False)
    diff_in = DiffusionInterface("u_1", "u_2", D1, D2, dim=1, time=False)

    diff_net_u_1 = instantiate_arch(
        input_keys=[Key("x")],
        output_keys=[Key("u_1")],
        cfg=cfg.arch.fully_connected,
    )
    diff_net_u_2 = instantiate_arch(
        input_keys=[Key("x")],
        output_keys=[Key("u_2")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        diff_u1.make_nodes()
        + diff_u2.make_nodes()
        + diff_in.make_nodes()
        + [diff_net_u_1.make_node(name="u1_network", jit=cfg.jit)]
        + [diff_net_u_2.make_node(name="u2_network", jit=cfg.jit)]
    )
```

### Step 3: Setting up the domain and assigning the boundary and PDE constraints



```python
    # make domain add constraints to the solver
    domain = Domain()
```

An L2 loss (default and can be modified) is then constructed from these constraints, which is used by the optimizer to minimize on. Specifying the constraints in this fashion is called soft-constraints.  

$$L = L_{BC} + L_{Residual}$$

**Boundary constraints:**

The boundary can be sampled using `PointwiseBoundaryConstraint` class. This will sample the entire boundary of the geometry we specify in the `geometry` argument, in this case, both the endpoints of the 1d line. A particular boundary of the geometry can be sub-sampled by using a particular criterion using the `criteria` parameter. For example, to sample the left end of `L1`, `criteria` is set to `Eq(x, 0)`. 

The desired values for the boundary condition are listed as a dictionary in `outvar` argument. For this problem,  we have `'u_1':0` at $x=0$ and `'u_2':100` at $x=2$. At $x=1$, we have the interface condition `'diffusion_interface_dirichlet_u_1_u_2':0` and `'diffusion_interface_neumann_u_1_u_2':0` that we defined earlier (i.e. $U_1=U_2$ and $D_1\frac{dU_1}{dx}=D_2\frac{dU_2}{dx}$). These dictionaries are then used when unrolling the computational graph (specified using the `nodes` argument) for training.

The number of points to sample on each boundary is specified using the `batch_size` parameter. 

**Equations to solve:** The Diffusion PDE we defined is enforced on all the points in the
interior of both the bars, `L1` and `L2`. We will use `PointwiseInteriorConstraint` class to sample points in the interior of the geometry. Again, the appropriate geometry is specified in the `geometry` argument; the equations to solve are specified as a dictionary input to `outvar` argument. These dictionaries are then used when unrolling the computational graph (specified using the `nodes` argument) for training.

For this problem we have the `'diffusion_u_1':0` and `'diffusion_u_2':0` for bars `L1` and `L2` respectively. The parameter `bounds`, determines the range for sampling the values for variables $x$ and $y$. The optional `lambda` parameter can be used to determine the weights for different losses. In this problem, we weight the loss on each point equally, and hence it is not used (defaults to 1) in this problem.

```python
    # sympy variables
    x = Symbol("x")

    # right hand side (x = 2) Pt c
    rhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"u_2": Tc},
        batch_size=cfg.batch_size.rhs,
        criteria=Eq(x, 2),
    )
    domain.add_constraint(rhs, "right_hand_side")

    # left hand side (x = 0) Pt a
    lhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"u_1": Ta},
        batch_size=cfg.batch_size.lhs,
        criteria=Eq(x, 0),
    )
    domain.add_constraint(lhs, "left_hand_side")

    # interface 1-2
    interface = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={
            "diffusion_interface_dirichlet_u_1_u_2": 0,
            "diffusion_interface_neumann_u_1_u_2": 0,
        },
        batch_size=cfg.batch_size.interface,
        criteria=Eq(x, 1),
    )
    domain.add_constraint(interface, "interface")

    # interior 1
    interior_u1 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"diffusion_u_1": 0},
        bounds={x: (0, 1)},
        batch_size=cfg.batch_size.interior_u1,
    )
    domain.add_constraint(interior_u1, "interior_u1")

    # interior 2
    interior_u2 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"diffusion_u_2": 0},
        bounds={x: (1, 2)},
        batch_size=cfg.batch_size.interior_u2,
    )
    domain.add_constraint(interior_u2, "interior_u2")
```

### Step 4: Adding Validators, Inferencers and Monitors

For this 1d bar problem where the conductivity is constant in each bar, the temperature varies linearly with the position inside the solid. The analytical solution can then be given as:

$$
\begin{align}
U_1 = xT_b + (1-x)T_a, && \text{when } 0 \leq x \leq 1 \\
U_2 = (x-1)T_c + (2-x)T_b, && \text{when } 1 \leq x \leq 2 \\
\end{align}
$$

where, 
$$
\begin{align}
T_a = U_1|_{x=0}, && T_c = U_2|_{x=2}, && \frac{\left(T_c + \left( D_1/D_2 \right)T_a \right)}{1+ \left( D_1/D_2 \right)}\\
\end{align}
$$

Now, let's create the validators. The validation data is added to the domain using the `PointwiseValidator` class. We use numpy to solve for the `u_1` and `u_2` based on the analytical expressions we showed above. The dictionary of generated numpy arrays (`invar_numpy` and `outvar_numpy`) for input and output variables and the appropriate nodes are used in the definition of the constructor. 

```python
    # validation data
    x = np.expand_dims(np.linspace(0, 1, 100), axis=-1)
    u_1 = x * Tb + (1 - x) * Ta
    invar_numpy = {"x": x}
    outvar_numpy = {"u_1": u_1}
    val = PointwiseValidator(nodes=nodes,invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val1")

    # make validation data line 2
    x = np.expand_dims(np.linspace(1, 2, 100), axis=-1)
    u_2 = (x - 1) * Tc + (2 - x) * Tb
    invar_numpy = {"x": x}
    outvar_numpy = {"u_2": u_2}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val2")
```

#### Adding Monitors 

Modulus library allows you to monitor desired quantities in Tensorboard as the simulation progresses and
assess the convergence. A `PointwiseMonitor` can be used to create such a feature. Examples of such quantities can be point values of variables, surface averages, volume averages or any derived quantities that can be formed using the variables being solved.

The variables are available as PyTorch tensors. We can perform tensor operations available in PyTorch to compute any desired derived quantity of our choice. 

In the code below, we create monitors for flux at the interface. The variable `u_1__x` represents the derivative of `u_1` in x-direction (two underscores (`__`) and the variable (`x`)). The same notation is used while handling other derivatives using the Modulus library. (eg. a neumann boundary condition of $\frac{dU_1}{dx}=0$ can be assigned as `'u_1__x':0` in the training domain for solving the same problem with an adiabatic/fixed flux boundary condition). 

The points to sample can be selected in a similar way as we did for specifying some of the interior constraints.

```python
    # make monitors
    invar_numpy = {"x": [[1.0]]}
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_1__x"],
        metrics={"flux_u1": lambda var: torch.mean(var["u_1__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_2__x"],
        metrics={"flux_u2": lambda var: torch.mean(var["u_2__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)
```

### Step 5: Hydra configuration

More information on the available configurations can be found in [Modulus Configuration](https://docs.nvidia.com/deeplearning/modulus/text/features/configuration.html).

```yaml
defaults :
  - modulus_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

arch:
    fully_connected:
        layer_size: 256

save_filetypes : "vtk,npz"

scheduler:
  decay_rate: 0.95
  decay_steps: 100

optimizer: 
  lr : 1e-4

training:
  rec_results_freq: 1000
  max_steps : 5000

batch_size:
  rhs: 2
  lhs: 2
  interface: 2
  interior_u1: 200
  interior_u2: 200
```

### Step 6: Putting everything together: Solver and training

 The solver can then be executed using the `solve` method. 

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

We are now ready to solve the PDEs !

This example is already saved for you and the code block below executes that script [`diffusion_bar.py`](../../source_code/diffusion_1d/diffusion_bar.py). You are encouraged to open the script and go through the code once before executing. Also, feel free to edit the parameters in the [`config.yaml`](../../source_code/diffusion_1d/conf/) file of the model and see its effect on the results.

In [ ]:
import os
os.environ["RANK"]="0"
os.environ["WORLD_SIZE"]="1"
os.environ["MASTER_ADDR"]="localhost"

In [ ]:
!python3 ../../source_code/diffusion_1d/diffusion_bar.py

### Visualizing the solution

Now, let's plot the temperature along the bar for the analytical and the neural network solution. A sample script to plot the results is shown below. If the training is complete, you should get the results like shown below. As we can see, our neural network solution and the analytical solution match almost exactly for this diffusion problem. 

<img src="images/image_diffusion_problem_bootcamp.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure()

network_dir = "./outputs/diffusion_bar/validators/"
data_1 = np.load(network_dir + "Val1.npz", allow_pickle=True)
data_2 = np.load(network_dir + "Val2.npz", allow_pickle=True)
data_1 = np.atleast_1d(data_1.f.arr_0)[0]
data_2 = np.atleast_1d(data_2.f.arr_0)[0]

plt.plot(data_1["x"][:, 0], data_1["pred_u_1"][:, 0], "--", label="u_1_pred")
plt.plot(data_2["x"][:, 0], data_2["pred_u_2"][:, 0], "--", label="u_2_pred")
plt.plot(data_1["x"][:, 0], data_1["true_u_1"][:, 0], label="u_1_true")
plt.plot(data_2["x"][:, 0], data_2["true_u_2"][:, 0], label="u_2_true")

plt.legend()
plt.show()

## Parameterized 1D Diffusion of Composite Bar

As we discussed in the introductory notebook, one important advantage of a PINN solver over traditional numerical methods is its ability to solve parameterized geometries and PDEs. This was initially proposed in the [paper](https://arxiv.org/abs/1906.02382) published by Sun et al. This allows us significant computational advantage, as one can now use PINNs to solve for multiple designs/cases in a single training. Once the training is complete, it is possible to run inference on several geometry/physical parameter combinations as a post-processing step without solving the forward problem again. 

To demonstrate the concept, we will train the same 1d diffusion problem, but now by parameterizing the conductivity of the first bar in the range $(5, 25)$. Once the training is complete, we can obtain the results for any conductivity value in that range saving us the time to train multiple models. 

### Case Setup - Parameterized

The definition of equations remains the same for this part. Since earlier, while defining the equations, we already defined the constants and coefficients of the PDE to be either numerical values or strings, this will allow us to parameterize <code>D1</code> by passing it as a string while calling the equations and making the neural network. Now let's start by creating the parameterized train domain. We will skip the parts common to the previous section and only discuss the changes. The complete script can be referred in <a href="../../source_code/diffusion_1d/diffusion_bar_parameterized.py" rel="nofollow"><code>diffusion_bar_parameterized.py</code></a>. 


### Step 1: Creating the geometry - Parameterized

In this problem, we will create the 1-dimensional geometry using `Line1D` class from the geometry module. The module also contains several 2d and 3d shapes like rectangle, circle, triangle, cuboids, sphere, torus, cones, tetrahedrons, etc. We will define the one dimensional line object using the two end-points. For the composite bar, we will create two separate bars as defined in the problem statement

```python
# params for domain
L1 = Line1D(0,1)
L2 = Line1D(1,2)
```

### Step 2: Adding Parameterized PDE and Neural Network nodes

Before starting out to create the nodes and domain, we will create the symbolic variable for the $D_1$ and also specify the range of variation for the variable. While the simulation runs, we will validate it against the same diffusion coefficient that we solved earlier i.e. $D_1=10$. 

```python
# params for domain
L1 = Line1D(0, 1)
L2 = Line1D(1, 2)

D1 = Symbol("D1")
D1_range = {D1: (5, 25)}
D1_validation = 1e1

D2 = 1e-1

Tc = 100
Ta = 0
Tb = (Tc + (D1 / D2) * Ta) / (1 + (D1 / D2))
Tb_validation = float(Tb.evalf(subs={D1: 1e1}))

print(Ta)
print(Tb)
print(Tc)
```

For training the parameterized model, we will have the symbolic parameters defined as inputs to both the neural networks in `diff_net_u_1` and `diff_net_u_2` viz. `'D1'` along with the usual x coordinate. The outputs remain the same as what we would have for any other non-parameterized simulation.

```python
@modulus.main(config_path="conf", config_name="config_param")
def run(cfg: ModulusConfig) -> None:
    # make list of nodes to unroll graph on
    diff_u1 = Diffusion(T="u_1", D="D1", dim=1, time=False)
    diff_u2 = Diffusion(T="u_2", D=D2, dim=1, time=False)
    diff_in = DiffusionInterface("u_1", "u_2", "D1", D2, dim=1, time=False)

    diff_net_u_1 = instantiate_arch(
        input_keys=[Key("x"), Key("D1")],
        output_keys=[Key("u_1")],
        cfg=cfg.arch.fully_connected,
    )
    diff_net_u_2 = instantiate_arch(
        input_keys=[Key("x"), Key("D1")],
        output_keys=[Key("u_2")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        diff_u1.make_nodes()
        + diff_u2.make_nodes()
        + diff_in.make_nodes()
        + [diff_net_u_1.make_node(name="u1_network", jit=cfg.jit)]
        + [diff_net_u_2.make_node(name="u2_network", jit=cfg.jit)]
    )
```

### Step 3: Adding Parameterized boundary and PDE Constraints

This part of the code is very similar to the non-parameterized version. The symbolic variables and the ranges that we described earlier need to be inputted to the `param_ranges` attribute of each boundary and internal constraints (`PointwiseBoundaryConstraint` and `PointwiseInteriorConstraint`)

```python
    # make domain add constraints to the solver
    domain = Domain()

    # sympy variables
    x = Symbol("x")

    # right hand side (x = 2) Pt c
    rhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"u_2": Tc},
        batch_size=cfg.batch_size.rhs,
        criteria=Eq(x, 2),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(rhs, "right_hand_side")

    # left hand side (x = 0) Pt a
    lhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"u_1": Ta},
        batch_size=cfg.batch_size.lhs,
        criteria=Eq(x, 0),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(lhs, "left_hand_side")

    # interface 1-2
    interface = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={
            "diffusion_interface_dirichlet_u_1_u_2": 0,
            "diffusion_interface_neumann_u_1_u_2": 0,
        },
        batch_size=cfg.batch_size.interface,
        criteria=Eq(x, 1),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interface, "interface")

    # interior 1
    interior_u1 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"diffusion_u_1": 0},
        bounds={x: (0, 1)},
        batch_size=cfg.batch_size.interior_u1,
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interior_u1, "interior_u1")

    # interior 2
    interior_u2 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"diffusion_u_2": 0},
        bounds={x: (1, 2)},
        batch_size=cfg.batch_size.interior_u2,
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interior_u2, "interior_u2")
```

### Step 4: Adding Validators and Monitors 

 Creating these domains is again similar to the previous section. For validation data, we need to create an additional key for the string <code>'D1'</code> in the <code>invar_numpy</code>. The value for this key can be in the range we specified earlier and which we would like to validate against. It is possible to create multiple validators if required, eg. different $D_1$ values. For the monitor domain, a similar <code>invar_numpy</code> is generated that has both the <code>'x'</code> and <code>'D1'</code> keys and appropriate arrays. 


```python
    # validation data
    x = np.expand_dims(np.linspace(0, 1, 100), axis=-1)
    u_1 = x * Tb_validation + (1 - x) * Ta
    invar_numpy = {"x": x}
    invar_numpy.update({"D1": np.full_like(invar_numpy["x"], D1_validation)})
    outvar_numpy = {"u_1": u_1}
    val = PointwiseValidator(nodes=nodes,invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val1")
    
    # make validation data line 2
    x = np.expand_dims(np.linspace(1, 2, 100), axis=-1)
    u_2 = (x - 1) * Tc + (2 - x) * Tb_validation
    invar_numpy = {"x": x}
    invar_numpy.update({"D1": np.full_like(invar_numpy["x"], D1_validation)})
    outvar_numpy = {"u_2": u_2}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val2")
    
    # make monitors
    invar_numpy = {"x": [[1.0]], "D1": [[D1_validation]]}
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_1__x"],
        metrics={"flux_u1": lambda var: torch.mean(var["u_1__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_2__x"],
        metrics={"flux_u2": lambda var: torch.mean(var["u_2__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)
```

### Step 5: Hydra configuration - Parameterized

More information on the available configurations can be found in [Modulus Configuration](https://docs.nvidia.com/deeplearning/modulus/text/features/configuration.html).

```yaml
defaults :
  - modulus_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

arch:
    fully_connected:
        layer_size: 256

save_filetypes : "vtk,npz"

scheduler:
  decay_rate: 0.95
  decay_steps: 200

optimizer: 
  lr : 1e-4

training:
  rec_results_freq: 1000
  max_steps : 10000

batch_size:
  rhs: 10
  lhs: 10
  interface: 10
  interior_u1: 400
  interior_u2: 400
```

### Step 6: Solver and Training

Now that we have the definitions for the various constraints and domains complete, we can form the solver and run the problem similarly to before. The code to do the same can be found below.

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

In [ ]:
!python3 ../../source_code/diffusion_1d/diffusion_bar_parameterized.py

### Visualizing the solution - Parameterized

The <code>.npz</code> arrays can be plotted similarly to the previous section to visualize the simulation output. You can see that we get the same answer as the analytical solution. You can try to run the problem in <code>eval</code> mode by changing the validation data and see how it performs for the other <code>D1</code> values as well. To run the model in evaluation mode (i.e. without training), you just need to <a href="https://docs.nvidia.com/deeplearning/modulus/text/features/configuration.html#run-modes" rel="nofollow">modify the config file</a>.  
<img src="images/image_diffusion_problem_bootcamp_parameterized.png" alt="Drawing" style="width:500px" />


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure()

network_dir = "./outputs/diffusion_bar_parameterized/validators/"
data_1 = np.load(network_dir + "Val1.npz", allow_pickle=True)
data_2 = np.load(network_dir + "Val2.npz", allow_pickle=True)
data_1 = np.atleast_1d(data_1.f.arr_0)[0]
data_2 = np.atleast_1d(data_2.f.arr_0)[0]

plt.plot(data_1["x"][:, 0], data_1["pred_u_1"][:, 0], "--", label="u_1_pred")
plt.plot(data_2["x"][:, 0], data_2["pred_u_2"][:, 0], "--", label="u_2_pred")
plt.plot(data_1["x"][:, 0], data_1["true_u_1"][:, 0], label="u_1_true")
plt.plot(data_2["x"][:, 0], data_2["true_u_2"][:, 0], label="u_2_true")

plt.legend()
plt.show()

You can see that at a fractional increase in computational time, we solved the PDE for $D_1$ ranging from (5, 25). This concept can easily be extended to more complicated problems, and this ability to solve parameterized problems comes very handy during design optimization and exploring the design space. For more examples of solving parameterized problems, please refer to <a href="https://docs.nvidia.com/deeplearning/modulus/index.html" rel="nofollow">Modulus User Documentation</a>


--- 

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---

# Licensing

Copyright © 2023 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.
